In [1]:
import tensorflow as tf
import os

In [2]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

In [3]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

In [4]:
print(os.getcwd())

C:\Users\aishw\01_PROJECT


In [5]:
base_dir = 'C:\\Users\\aishw\\01_PROJECT\\data'

In [6]:
train_generator = datagen.flow_from_directory(base_dir,target_size=(IMAGE_SIZE,IMAGE_SIZE),batch_size=BATCH_SIZE,subset='training')

Found 76 images belonging to 2 classes.


In [7]:
val_generator = datagen.flow_from_directory(base_dir,target_size=(IMAGE_SIZE,IMAGE_SIZE),batch_size=BATCH_SIZE,subset='validation')

Found 18 images belonging to 2 classes.


In [8]:
print(train_generator.class_indices)

{'covid': 0, 'normal': 1}


In [9]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))

In [10]:
labels

'covid\nnormal'

In [11]:
with open('covid_labels.txt','w+') as f:
    f.write(labels)

In [12]:
IMG_SHAPE =(IMAGE_SIZE,IMAGE_SIZE,3)

In [13]:
base_model =tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights = 'imagenet')

In [14]:
base_model.trainable = False

In [15]:
model = tf.keras.Sequential([
                             base_model,
                             tf.keras.layers.Conv2D(32,4, activation='relu'),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.GlobalAveragePooling2D(),
                             tf.keras.layers.Dense(2,activation='softmax')
                             ])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 4, 4, 32)          655392    
_________________________________________________________________
dropout (Dropout)            (None, 4, 4, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 66        
Total params: 2,913,442
Trainable params: 655,458
Non-trainable params: 2,257,984
_________________________________________________________________


In [17]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [22]:
epochs=40

In [19]:
import PIL

In [23]:
history=model.fit(train_generator,epochs=epochs,validation_data=val_generator)

Epoch 1/40
2/2 [==============================] - 1s 599ms/step - loss: 8.8800e-05 - accuracy: 1.0000 - val_loss: 4.5511e-05 - val_accuracy: 1.0000
Epoch 2/40
2/2 [==============================] - 1s 487ms/step - loss: 1.2025e-04 - accuracy: 1.0000 - val_loss: 4.5458e-05 - val_accuracy: 1.0000
Epoch 3/40
2/2 [==============================] - 1s 465ms/step - loss: 1.3557e-04 - accuracy: 1.0000 - val_loss: 4.5405e-05 - val_accuracy: 1.0000
Epoch 4/40
2/2 [==============================] - 1s 465ms/step - loss: 8.9073e-05 - accuracy: 1.0000 - val_loss: 4.5313e-05 - val_accuracy: 1.0000
Epoch 5/40
2/2 [==============================] - 1s 469ms/step - loss: 6.3516e-05 - accuracy: 1.0000 - val_loss: 4.5114e-05 - val_accuracy: 1.0000
Epoch 6/40
2/2 [==============================] - 3s 2s/step - loss: 3.4873e-05 - accuracy: 1.0000 - val_loss: 4.4929e-05 - val_accuracy: 1.0000
Epoch 7/40
2/2 [==============================] - 1s 506ms/step - loss: 9.6097e-05 - accuracy: 1.0000 - val_loss: 4

In [21]:
model.evaluate(val_generator)

1/1 [==============================] - 0s 1ms/step - loss: 4.5458e-05 - accuracy: 1.0000


[4.5458367821993306e-05, 1.0]

In [25]:
model.save('covid_model.h5')